In [36]:
# Week1 Assignment
# Create notebook

In [1]:
import pandas as pd
import numpy as np

print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [2]:
# Week 3 Assignment
# Segmenting and Clustering Neighborhoods in Toronto

In [3]:
# 1. Start by creating a new Notebook for this assignment.
# 2. Use the Notebook to build the code to scrape the following Wikipedia page

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [4]:
# 3. To create the dataframe:

# 3.1 Ignore cells with a borough that is Not Assigned
df = df[df.Borough != 'Not assigned']
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [26]:
# 3.2 One neighborhood can exist in one postal code area. 
group_df = df.groupby('Postal code')
df_grouped = group_df.agg({'Borough':'first', 'Neighborhood': ','.join}).reset_index()
df_grouped.head()

# Notice that Wikipidea is already doing the Neighborhood merge with '/'
# Convert '/' to ',' to match sample format
df_grouped['Neighborhood'] = df_grouped['Neighborhood'].apply(lambda x : x.replace('/', ','))
df_grouped.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [27]:
# Print the number of rows in the dataframe
df_grouped.shape

(103, 3)

In [28]:
# Use the Geocoder package or the csv file to create data frame
# Commented out this cell because data is not retrieveable using Geocoder package

#import geocoder # import geocoder


# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format('M1B'))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

# print('{}, {}'.format(latitude, longitude))

In [212]:
# Read data from csv file to get geo info of postal code

df_geocoder = pd.read_csv('http://cocl.us/Geospatial_data').rename({'Postal Code': 'Postal code'}, axis='columns')
df_geocoder.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [214]:
# Merge two data frames
df_canada = df_grouped.merge(df_geocoder, on='Postal code')
df_canada.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [215]:
# Look at only toronto data
df_toronto = df_canada[df_canada['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_toronto.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [216]:
df_toronto.shape

(39, 5)

In [37]:
import requests

# hide some info for privilege
CLIENT_ID = '******' # your Foursquare ID
CLIENT_SECRET = '******' # your Foursquare Secret
VERSION = '20200501' # Foursquare API version

In [222]:
# Test run FourSquare API

LIMIT = 100
radius = 500

neighborhood_lat = df_toronto.loc[0, 'Latitude']
neighborhood_lon = df_toronto.loc[0, 'Longitude']


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat, 
    neighborhood_lon, 
    radius, 
    LIMIT)

response = requests.get(url).json()
response

{'meta': {'code': 200, 'requestId': '5eacccdb40a7ea001bea5259'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [221]:
# Play around data coming back from FourSquare API

from pandas import json_normalize

results = response['response']['groups'][0]['items']
results

venues = results
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,Glen Stewart Ravine,Other Great Outdoors,43.676300,-79.294784
2,Glen Stewart Park,Park,43.675278,-79.294647


In [224]:
# Method that calls FourSquare API iteratively for every neighborhood and create the venue lists
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    venue_list = []
    for name, lat, lon in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lon, 
            radius, 
            limit)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        venue_list.append([(
                name,
                lat,
                lon,
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results
        ])
    return venue_list

In [225]:
# Call the getNearbyVenues to get toronto neighborhood venue list
venue_list = getNearbyVenues(df_toronto['Neighborhood'], df_toronto['Latitude'], df_toronto['Longitude'])
venue_list

[[('The Beaches',
   43.67635739999999,
   -79.2930312,
   'Glen Manor Ravine',
   43.67682094413784,
   -79.29394208780985,
   'Trail'),
  ('The Beaches',
   43.67635739999999,
   -79.2930312,
   'The Big Carrot Natural Food Market',
   43.678879,
   -79.297734,
   'Health Food Store'),
  ('The Beaches',
   43.67635739999999,
   -79.2930312,
   'Grover Pub and Grub',
   43.679181434941015,
   -79.29721535878515,
   'Pub'),
  ('The Beaches',
   43.67635739999999,
   -79.2930312,
   'Upper Beaches',
   43.68056321147582,
   -79.2928688743688,
   'Neighborhood')],
 [('The Danforth West , Riverdale',
   43.6795571,
   -79.352188,
   'MenEssentials',
   43.677820068604575,
   -79.35126543045044,
   'Cosmetics Shop'),
  ('The Danforth West , Riverdale',
   43.6795571,
   -79.352188,
   'Pantheon',
   43.67762124481265,
   -79.35143390043564,
   'Greek Restaurant'),
  ('The Danforth West , Riverdale',
   43.6795571,
   -79.352188,
   'Cafe Fiorentina',
   43.677743,
   -79.350115,
   'Italia

In [226]:
# Build venue dataframe from venue list
df_venue = pd.DataFrame([item for venues_list in venue_list for item in venues_list], columns=['Name', 'Latitude', 'Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'])
df_venue.head(10)

,Name,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West , Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
5,"The Danforth West , Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
6,"The Danforth West , Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
7,"The Danforth West , Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
8,"The Danforth West , Riverdale",43.679557,-79.352188,La Diperie,43.677530,-79.352295,Ice Cream Shop
9,"The Danforth West , Riverdale",43.679557,-79.352188,Moksha Yoga Danforth,43.677622,-79.352116,Yoga Studio


In [227]:
# Group venues by neighborhood
df_venue.groupby('Name').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Name,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton , Parkdale Village , Exhibition Place",23,23,23,23,23,23
Business reply mail Processing CentrE,18,18,18,18,18,18
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",17,17,17,17,17,17
Central Bay Street,63,63,63,63,63,63
Christie,17,17,17,17,17,17
Church and Wellesley,78,78,78,78,78,78
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36


In [228]:
# Create one hot dataframe

toronto_onehot = pd.get_dummies(df_venue[['Venue Category']], prefix='', prefix_sep='')
toronto_onehot['Name'] = df_venue['Name'] 
toronto_onehot.head()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Name
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,The Beaches
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,The Beaches
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,The Beaches
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,The Beaches
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"The Danforth West , Riverdale"


In [229]:
# Rearrange table and put neighborhood name left-most

cols = toronto_onehot.columns.tolist()
columns = cols[-1:] + cols[:-1]
toronto_onehot = toronto_onehot[columns]
toronto_onehot.head()

,Name,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West , Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [230]:
toronto_grouped = toronto_onehot.groupby('Name').mean().reset_index()
toronto_grouped

,Name,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business reply mail Processing CentrE,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556
3,"CN Tower , King and Spadina , Railway Lands , ...",0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.015873
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641
7,"Commerce Court , Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.00,...,0.027778,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [259]:
# Get the top 10 venues for every neighborhood

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Name'] = toronto_grouped['Name']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Bakery,Beer Bar,Restaurant,Seafood Restaurant,Clothing Store,Beach
1,"Brockton , Parkdale Village , Exhibition Place",Café,Breakfast Spot,Coffee Shop,Nightclub,Burrito Place,Restaurant,Italian Restaurant,Stadium,Intersection,Bar
2,Business reply mail Processing CentrE,Yoga Studio,Pizza Place,Spa,Fast Food Restaurant,Auto Workshop,Farmers Market,Burrito Place,Smoke Shop,Recording Studio,Garden
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport Terminal,Airport,Bar,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry,Boutique
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bar,Thai Restaurant,Salad Place,Ice Cream Shop,Burger Joint,Bubble Tea Shop


In [260]:
# Run kmeans to tag every neighborhood

from sklearn.cluster import KMeans

kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Name', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([4, 4, 4, 4, 4, 6, 4, 4, 1, 8], dtype=int32)

In [261]:
# Merge clustering data with initial geolocation data

neighborhoods_venues_sorted
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Name'), on='Neighborhood')

toronto_merged.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Neighborhood,Pub,Health Food Store,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Yoga Studio
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188,4,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Frozen Yogurt Shop,Ice Cream Shop,Spa,Japanese Restaurant,Juice Bar
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572,1,Sandwich Place,Fast Food Restaurant,Gym,Park,Brewery,Burrito Place,Restaurant,Pub,Pizza Place,Movie Theater
3,M4M,East Toronto,Studio District,43.659526,-79.340923,4,Café,Coffee Shop,Gastropub,Brewery,Bakery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,7,Park,Construction & Landscaping,Bus Line,Swim School,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [262]:
# Get Toronto geolocation

from geopy.geocoders import Nominatim

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

"Toronto latitude: {}, longitude: {}".format(latitude, longitude)

'Toronto latitude: 43.6534817, longitude: -79.3839347'

In [263]:
# Display different clusters in map

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters